**处理用户好友数据**

In [1]:
import pandas as pd

import numpy as np
import scipy.sparse as ss
import scipy.io as sio

#保存数据
import pickle

from sklearn.preprocessing import normalize

In [3]:
#读取训练集和测试集中出现过的事件列表
userIndex = pickle.load(open("PE_userIndex.pkl", 'rb'))
n_users = len(userIndex)

print("number of users in train & test :%d" % n_users)

number of users in train & test :3391


In [5]:
#用户-事件关系矩阵
userEventScores = sio.mmread("PE_userEventScores")

#后续用于将用户朋友参加的活动影响到用户
eventsForUser = pickle.load(open("PE_eventsForUser.pkl", 'rb'))

In [8]:
#读取数据

"""
  找出某用户的那些朋友
  1)如果你有更多的朋友，可能你性格外向，更容易参加各种活动
  2)如果你朋友会参加某个活动，可能你也会跟随去参加一下
"""
 
#用户有多少个朋友
numFriends = np.zeros((n_users))
userFriends = ss.dok_matrix((n_users, n_users))
user_friends_dpath = "C:/Users/Lzg/Desktop/data/w4/user_friends.csv"    
fin = open(user_friends_dpath, 'rb')
#字段：user，friends
fin.readline()                # skip header

#ln = 0
for line in fin:  #对每个用户        
    cols = line.decode().strip().split(",")
    user = str(cols[0])    #user
    
    if  user in userIndex :   #该用户在训练集和测试集的用户列表中
        friends = cols[1].split(" ")  #friends
        i = userIndex[user]       #该用户的索引
        numFriends[i] = len(friends)
        for friend in friends:  #该用户的每个朋友
            str_friend = str(friend)
            if  str_friend in userIndex:  #如果朋友也在训练集或测试集中出现
                j = userIndex[str_friend]   #朋友的索引
            
                # the objective of this score is to infer the degree to
                # and direction in which this friend will influence the
                # user's decision, so we sum the user/event score for
                # this user across all training events.
            
                #userEventScores为用户对活动的打分（interested - not interseted）
                #在Users-Events.ipynb中计算好了
                eventsForUser = userEventScores.getrow(j).todense()
            
                #所有朋友参加活动的数量（平均频率）
                score = eventsForUser.sum() / np.shape(eventsForUser)[1]
                userFriends[i, j] += score
                userFriends[j, i] += score
            
fin.close()
    

#用户的朋友数目
# 归一化数组
sumNumFriends = numFriends.sum(axis=0)
numFriends = numFriends / sumNumFriends
sio.mmwrite("UF_numFriends", np.matrix(numFriends))

#
userFriends = normalize(userFriends, norm="l2", axis=0, copy=False)
sio.mmwrite("UF_userFriends", userFriends)

In [9]:
numFriends

array([1.33462794e-04, 1.23332486e-03, 2.14398063e-04, ...,
       7.50393219e-05, 1.42038717e-04, 6.40514212e-05])